<a href="https://colab.research.google.com/github/PhilipPfeffer/haptic_vest/blob/main/image_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow import keras
from pycocotools.coco import COCO     # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py

In [2]:
# BORROWED FROM DATA_PIPELINE
# TODO: CALL FROM DATA_PIPELINE
def create_coco(path_to_instances_file, datasetName='val2017'):
  annotations=f'{path_to_instances_file}instances_{datasetName}.json'
  print(annotations)
  
  # This line loads annotations from an instances_{datasetName}.json file into a COCO object so we can use the pycoco API
  coco = COCO(annotations)
  return coco

#USAGE
dataset_name='train2017'
path_to_instances_file ='/content/drive/My Drive/Haptic Vest/coco_dataset/annotations/'
coco = create_coco(path_to_instances_file, dataset_name)

/content/drive/My Drive/Haptic Vest/coco_dataset/annotations/instances_train2017.json
loading annotations into memory...
Done (t=17.42s)
creating index...
index created!


Download the images

In [3]:
import skimage
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import resize
import os

In [4]:
dataset ='/content/drive/My Drive/Haptic Vest/dataset_person_car'

In [5]:
train_ids = np.loadtxt(dataset, dtype=(int, int))

In [6]:
x_train_ids, y_train_ids = train_ids[:,0], train_ids[:,1]
assert(x_train_ids.shape == y_train_ids.shape)
x_train_ids[0], y_train_ids[0]

(262145, 1)

In [7]:
# download_imgs_dir = '/content/drive/My Drive/Haptic Vest/person_car_other_imgs'
# coco.download(download_imgs_dir, x_train_ids)

In [8]:
# image_count = 0
# for entry in os.scandir(download_imgs_dir):
#   image_count += 1
#   # print(entry.path)
# print(f"{image_count} images downloaded")

In [9]:
# # Process from downloads directory

# processed_count = 0
# for entry in os.scandir(download_imgs_dir):
#   processed_count += 1
#   img = io.imread(entry.path)
#   img_width = img.shape[0]
#   img_height = img.shape[1]
#   processed = img
  
#   # Greyscale
#   if len(img.shape) != 2:
#     processed = rgb2gray(processed)
  
#   # Square (centre-crop)
#   if img_height != img_width:
#     if img_width < img_height:
#       height_start = int((img_height - img_width) / 2)
#       processed = img[:, height_start:height_start+img_width]
#     else:
#       width_start = int((img_width - img_height) / 2)
#       processed = img[width_start:width_start+img_height,:]
  
#   # Resize (96x96)
#   if img_height != 96:
#     processed = resize(processed, (96,96))

#   # io.imshow(processed)
#   io.imsave(entry.path, processed)

# assert(processed_count == image_count)

In [10]:
# # Example image
# for i, entry in enumerate(os.scandir(download_imgs_dir)):
#   if i == 400:
#     img = io.imread(entry.path)
#     io.imshow(entry.path)
#     print(img.shape)
#     print(entry.path)
#     break

#Process from URL
Better than from downloaded images because the original RGB images take up a lot of space.

In [7]:
download_imgs_dir = '/content/drive/My Drive/Haptic Vest/person_car_other_imgs_processed'

In [9]:
def count_processed_images(download_imgs_dir, total=0):
  image_count = 0
  for entry in os.scandir(download_imgs_dir):
    image_count += 1
    # print(entry.path)
  print(f"{image_count}/{total} images downloaded")

count_processed_images(download_imgs_dir, x_train_ids.shape[0])

77033/77650 images downloaded


In [10]:
already_processed_filenames = [os.path.basename(entry) for entry in os.scandir(download_imgs_dir)]
processed_ids = [int(name[:-4]) for name in already_processed_filenames]

In [11]:
set_difference = set(x_train_ids) - set(processed_ids)
remaining_ids = list(set_difference)
print(len(remaining_ids))
print(remaining_ids[:10])

0
[]


In [12]:
# Process from URLs

processed_count_URL = 0
imgs = coco.loadImgs(remaining_ids)
checked_already_processed = 0
for img_info in imgs:
  processed_count_URL += 1

  img = io.imread(img_info['coco_url'])
  img_filename = img_info['file_name']
  img_width = img_info['width']
  img_height = img_info['height']
  processed = img
  
  # if img_filename in already_processed_filenames:
  #   checked_already_processed += 1
  #   if checked_already_processed % 1000:
  #     print(f"{checked_already_processed} of the images checked have already been processed.")
  #   continue

  # Greyscale
  if len(img.shape) != 2:
    processed = rgb2gray(processed)
  
  # Square (centre-crop)
  if img_height != img_width:
    if img_width < img_height:
      height_start = int((img_height - img_width) / 2)
      processed = processed[height_start:height_start+img_width,:]
    else:
      width_start = int((img_width - img_height) / 2)
      processed = processed[:,width_start:width_start+img_height]
  
  # Resize (96x96)
  if img_height != 96:
    processed = resize(processed, (96,96))

  io.imsave(download_imgs_dir+'/' + img_filename, processed);
  
  if processed_count_URL % 1000 == 0:
    print(f"\n{processed_count_URL} images processed")

print(f"\nDone!\n{processed_count_URL} images processed")


Done!
0 images processed
